In [17]:
#read data
import pandas as pd
import numpy as np
train_data = pd.read_csv("house-prices-advanced-regression-techniques/train.csv")
test_data = pd.read_csv("house-prices-advanced-regression-techniques/test.csv")

In [18]:
#clean data
train_data =  pd.get_dummies(train_data)
train_data = train_data.fillna(0)
test_data =  pd.get_dummies(test_data)
test_data = test_data.fillna(0)
#print(clear_data.columns)
train_data.to_csv("train_data.txt", sep="\t", index=False)
test_data.to_csv("test_data.txt", sep="\t", index=False)

In [19]:
def normalize(column):
    return (column - column.min()) / (column.max() - column.min())

train_data = train_data.astype(float)
test_data  = test_data.astype(float)

train_data.iloc[:,1:] = train_data.iloc[:,1:].apply(normalize)
test_data.iloc[:,1:]  = test_data.iloc[:,1:].apply(normalize)

train_data.to_csv("train_data.txt", sep="\t", index=False)
test_data.to_csv("test_data.txt", sep="\t", index=False)